# Encontrar los valores medios de los histogramas y matrices

----
**Table of contents**<a id='toc0_'></a>    
- [General](#toc1_1_)    
- [Funciones](#toc1_2_)    
- [_Main_](#toc1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

----
## <a id='toc1_1_'></a>[General](#toc0_)

In [54]:

import pickle
import os
import sys
import numpy as np
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm as tqdm
import itertools as it

general_directory=os.getcwd()+'/Data/'
cases=['allt','hw','no','mo']
st=['st','unst']

----
## <a id='toc1_2_'></a>[Funciones](#toc0_)

In [63]:
def get_days(period):
    if not os.path.isdir(general_directory): os.mkdir('Data')
    if not os.path.isdir(general_directory+'Days'): os.mkdir(general_directory+'Days')
    days=[i for i in os.listdir(general_directory+'Saves')]
    ndays=[]
    for day in days:
        try:
            int(day[0])
            if day[:4] not in ndays:
                ndays.append(day[:4])
        except:
            pass
    days=ndays
    days.sort
    if period=='week':
        days=[days[i][0:10] for i in range(len(days)) if all([(i-j)%7 != 0 for j in [3,4]])]
    if period=='weekend':
        days=[days[i][0:10] for i in range(len(days)) if any([(i-j)%7 == 0 for j in [3,4]])]
    return days

In [56]:
def save_dataP(name,dictionary,location=general_directory):

    with open(location+'Saves/'+ name+'.pkl', 'wb') as tf:
        pickle.dump(dictionary,tf)

In [76]:
def load_hist(day,case,st):
    with open(general_directory+'Saves/'+day+'_'+case+'_'+st+'_hist.pkl','r+b') as pk:
        hist=pickle.load(pk)
    return hist


def load_mat(day,case,st):
    m=np.genfromtxt(general_directory+'Saves/'+day+'_'+case+'_'+st+'_mat.csv',delimiter=',')
    return m


In [77]:
def mean(case,st,period):
    Mm=0
    Mh=0
    with tqdm(desc="Mean of cases "+case,total=2*len(days)) as pbar:
        for day in days:
            Mm+=load_mat(day,case,st)
            pbar.update()
        np.savetxt(general_directory+'/Saves/Meanm_'+case+'_'+st+'_'+period+'.csv',Mm/N,delimiter=',')
        for day in days:
            Mh+=load_hist(day,case,st)
            pbar.update()
        save_dataP('Meanh_'+case+'_'+st+'_'+period,Mh/N)

----
## <a id='toc1_3_'></a>[_Main_](#toc0_)

In [71]:


def main(period):
    days=get_days(period)
    N=len(days)

    if N==0: print('No data yet')
    else:
        with ProcessPoolExecutor(max_workers=int(multiprocessing.cpu_count()/2)) as ex:
            for i in st:
                list(ex.map(mean,cases,[i for case in cases],[period for case in cases]))
 

In [60]:
       

if __name__=="__main__":
    
    main(period)

TypeError: main() missing 1 required positional argument: 'period'

In [80]:
days=get_days('weekend')
N=len(days)
for case in cases:
    for s in st:
        mean(case,s,'weekend')

Mean of cases mo: 100%|████████████████████████████████████████████████████████████████| 20/20 [00:14<00:00,  1.42it/s]
